# Merging All Datasets in One Graph

As usual, importing the needed libraries is the first step

In [3]:
import plotly.plotly as py
import pandas as pd
import numpy as np
#import cufflinks as cf 
#cf.go_offline()
import plotly.graph_objs as go
from plotly.offline import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode( connected = True )
import urllib.request
import json
from pandas.io.json import json_normalize

## A. Reading Data

In [4]:
#Calling Live Data from API 
with urllib.request.urlopen("https://www.numbeo.com/api/rankings_by_city_historical?api_key=aml7iv7ua8pnxo&section=1") as url:
    dataCostOfLiving = json.load(url)
    print(dataCostOfLiving)

{'2012': [{'country': 'Norway', 'city_name': 'Trondheim', 'cpi_and_rent_index': 142.213775943675, 'rent_index': 59.1643129137286, 'purchasing_power_incl_rent_index': 67.0079542854664, 'restaurant_price_index': 160.230357098712, 'groceries_index': 193.936491910627, 'city_id': 5970, 'cpi_index': 188.910503280187}, {'country': 'Norway', 'city_name': 'Stavanger', 'cpi_and_rent_index': 137.762589838008, 'rent_index': 78.0809547405555, 'purchasing_power_incl_rent_index': 78.4958678639918, 'restaurant_price_index': 201.163181235453, 'groceries_index': 147.735435337591, 'city_id': 7409, 'cpi_index': 171.32014625638}, {'country': 'Switzerland', 'city_name': 'Zurich', 'cpi_and_rent_index': 124.674876830621, 'rent_index': 74.5835847462938, 'purchasing_power_incl_rent_index': 142.77480774886, 'restaurant_price_index': 138.786508076975, 'groceries_index': 143.698374998437, 'city_id': 6379, 'cpi_index': 152.840012679265}, {'country': 'Norway', 'city_name': 'Oslo', 'cpi_and_rent_index': 117.969883353

In [5]:
# Reading Data
homelessness = pd.read_csv('2007-2016-Homelessnewss-USA.csv')
population =  pd.read_csv('Population-by-state.csv')
race = pd.read_excel('2015 Racial Data _ State.xls','raw_data.csv')

## B. Cleaning data

### (1) Population DataSet:

 Dropping and renaming columns 

In [6]:
# Cleaning datasets 
# Begining with Population DataSet 
# 1. taking the first row and making it the header
population.columns = population.iloc[0]
# 2. removing the 0 row
population.drop(0,axis=0, inplace=True)
# 3. Removing Unneeded columns
population.drop(['Id', 'Id2','April 1, 2010 - Census', 'April 1, 2010 - Estimates Base'], axis=1, inplace=True)
# 4. Renaming the columns, titles are too long
population.columns = ['State','pop2010','pop2011','pop2012','pop2013','pop2014','pop2015','pop2016']
population.head(3)

,State,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016
1,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300
2,Alaska,714031,722713,731089,736879,736705,737709,741894
3,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071


### (2) Homelessness DataSet: <br/>
Converting to integers, dropping a column

In [7]:
## Since every column is an object ,, we need to convert Count and Year to integers
# Cleaning datasets 
# Homelessness DataSet 
# 1. converting Count to String (remove the comma) and then to an integer
homelessness['Count'] = homelessness['Count'].str.replace(',', '')
homelessness['Count'] = pd.to_numeric(homelessness['Count'])
# 2. Removing Unneeded columns
## Continuum of Care (CoC) 
homelessness.drop(['CoC Number','CoC Name','Measures'], axis=1, inplace=True)
# 3. Converting Year to dateTime and then to an integer
## so that we can check for any year we want 
homelessness['Year'] = pd.to_datetime(homelessness['Year'])
homelessness['Year'] = homelessness['Year'].dt.year
homelessness.head(20)

,Year,State,Count
0,2007,AK,224
1,2007,AK,696
2,2007,AK,278
3,2007,AK,187
4,2007,AK,842
5,2007,AK,589
6,2007,AK,253
7,2007,AK,974
8,2007,AK,37
9,2007,AK,132


 <br/>
Copying the dataset, renaming the columns
<br/>
** Defining a list of the USA states. 

In [8]:
StatesFullNames = [ 'Alaska', 'Alabama', 'Arkansas', 'Arizona', 
'California', 'Colorado', 'Connecticut', 'District of Columbia', 
'Delaware', 'Florida', 'Georgia','Guam','Hawaii', 'Iowa', 
'Idaho','Illinois', 'Indiana', 'Kansas', 'Kentucky',
'Louisiana','Massachusetts', 'Maryland', 'Maine', 'Michigan', 
'Minnesota', 'Missouri', 'Mississippi', 'Montana', 
'North Carolina', 'North Dakota', 'Nebraska', 'New Hampshire', 
'New Jersey','New Mexico', 'Nevada', 'New York', 'Ohio', 
'Oklahoma', 'Oregon', 'Pennsylvania', 'Puerto Rico', 
'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 
'Texas','Utah', 'Virginia', 'Virgin Islands','Vermont', 
'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']

## making a copy of the homeless dataset to use it in our map
homelessnessCopy1 = homelessness.copy()
# Removing the unneeded column in our map
## we want to show in our map, the state & # of homeless people (count)
#homelessnessCopy1.drop(['Measures'], axis=1, inplace=True)
homelessnessCopy1.columns = ['Year','StateAbbrev','CountHomeless']


### (3) Data from API:  <br/>
-Normalizing the live data be used as a dataset<br/>
-Selecing the cost of living for each USA state for 2015 & dropping remaining columns 

In [9]:

# Normalizing the live data called from API to be used as a dataset
CostOfLivingAll = json_normalize(dataCostOfLiving)
CostOfLivingAll
# Selecing the cost of living for each USA state for 2015 & dropping remaining columns
CostOfLivingCities2015 = json_normalize(dataCostOfLiving['2015-mid'])
CostOfLivingCities2015isUSA = CostOfLivingCities2015['country']=='United States'
CostOfLivingCities2015USA = CostOfLivingCities2015[CostOfLivingCities2015isUSA]
CostOfLivingCities2015USA.drop(['restaurant_price_index','rent_index','purchasing_power_incl_rent_index','groceries_index','cpi_and_rent_index','city_id','country'],axis=1, inplace=True )
CostOfLivingCities2015USA.head()

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,city_name,cpi_index
18,"San Francisco, CA",100.275779
19,"New York, NY",100.000000
21,"Honolulu, HI",99.033270
23,"Washington, DC",97.717446
28,"Brooklyn, NY",96.056669


In [10]:
# Removing the name of the cities 
CostOfLivingCities2015USA['city_name'] = CostOfLivingCities2015USA['city_name'].apply(lambda x: x.split(',')[-1])
CostOfLivingCities2015USA.head(3)

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,city_name,cpi_index
18,CA,100.275779
19,NY,100.000000
21,HI,99.033270


In [11]:
# Selecting states with highest 'cpi_index=CostOfLiving' GroupBy CityName & nLargest Cpi_Index
CostOfLivingCities2015USA = CostOfLivingCities2015USA.groupby('city_name')['cpi_index'].apply(lambda grp: grp.nlargest(1)).reset_index()
# Sorting the values (cost of living) from largest to smallest
CostOfLivingCities2015USA = CostOfLivingCities2015USA.sort_values('cpi_index', ascending=False)[['cpi_index', 'city_name']].reset_index()
CostOfLivingCities2015USA.head(6)


,index,cpi_index,city_name
0,3,100.275779,CA
1,24,100.000000,NY
2,9,99.033270,HI
3,6,97.717446,DC
4,0,91.836137,AK
5,34,91.700292,WA


In [12]:
# Dropping the new added index 
CostOfLivingCities2015USA.drop(['index'],axis=1, inplace=True )
# renaming the columns (Easier to understand)
CostOfLivingCities2015USA.columns = ['CostOfLiving','StateAbbrev']
##The names column in StateAbbrev has extra white-space surrounding the text.
CostOfLivingCities2015USA['StateAbbrev'] = CostOfLivingCities2015USA['StateAbbrev'].str.strip()

CostOfLivingCities2015USA.head()

,CostOfLiving,StateAbbrev
0,100.275779,CA
1,100.000000,NY
2,99.033270,HI
3,97.717446,DC
4,91.836137,AK


In [13]:
CostOfLivingCities2015USA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
CostOfLiving    36 non-null float64
StateAbbrev     36 non-null object
dtypes: float64(1), object(1)
memory usage: 656.0+ bytes


-Removing the first index column instead of having <br/>
            Year    2007    2008    2009<br/>
            State <br/>
We want to have 
<br/> State 2007  2008  2009
<br/> and so on
<br/><br/>

-Adding the list of State names to HomelessnessCopy1 as a new column

In [14]:
# Converting the Years rows to columns and setting the # of homeless to max to avoid duplicates
homelessnessCopy1 = homelessnessCopy1.groupby(['StateAbbrev', 'Year'])['CountHomeless'].max().unstack()

homelessnessCopy1 = homelessnessCopy1.reset_index().rename_axis(None).rename_axis(None, axis=1)
# Adding the States full names column to the dataset
homelessnessCopy1['State'] = pd.Series(StatesFullNames)
homelessnessCopy1.head() 

,StateAbbrev,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,State
0,AK,974,1023,1267,1231,1223,1147,1122,1023,1208,1105,Alaska
1,AL,2104,2104,2273,2273,1950,1707,1469,1329,1153,1228,Alabama
2,AR,1822,1811,1425,1425,1276,1873,1678,1074,830,808,Arkansas
3,AZ,8448,7189,7889,6999,5831,6485,5889,5918,5631,5702,Arizona
4,CA,47862,47862,33243,33243,34622,31553,35524,34393,41174,43854,California


Dropping # of homeless columns in the years (2007, 2014 and 2016)

In [15]:
homelessnessCopy1.drop([2007,2008,2009,2010,2011,2012,2013,2014,2016], axis=1, inplace=True)
homelessnessCopy1.head()

,StateAbbrev,2015,State
0,AK,1208,Alaska
1,AL,1153,Alabama
2,AR,830,Arkansas
3,AZ,5631,Arizona
4,CA,41174,California


### (4) Race DataSet:

In [16]:
race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 9 columns):
Location                                  52 non-null object
White                                     52 non-null float64
Black                                     51 non-null float64
Hispanic                                  52 non-null float64
Asian                                     47 non-null float64
American Indian/Alaska Native             9 non-null float64
Native Hawaiian/Other Pacific Islander    13 non-null float64
Two Or More Races                         45 non-null float64
Total                                     52 non-null int64
dtypes: float64(7), int64(1), object(1)
memory usage: 3.7+ KB


-Dropping columns with many null values & uneeded columns  <br/>
(Note: columns with many null values do not have much effect on the data here or especially the chosen states that have higher number & percentage of homeless ) <br/>
-Replacing the null values in the two columns that null values with the mean values of the columns

In [17]:
## there are NaN values--> must be replaced by 0 
## will have to delete the [ American Indian/Alaska Native & Native Hawaiian/Other Pacific Islander & Two Or More Races & Total]
## since there is no enough data about them
## No need for the 2 
race.drop(['Native Hawaiian/Other Pacific Islander', 'American Indian/Alaska Native','Total', 'Two Or More Races'], axis=1, inplace=True)

# replaceing NaN with the mean values of the columns that have null values
race['Black'].fillna(race['White'].mean(), inplace = True)
race['Asian'].fillna(race['Asian'].mean(), inplace = True)

race.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 5 columns):
Location    52 non-null object
White       52 non-null float64
Black       52 non-null float64
Hispanic    52 non-null float64
Asian       52 non-null float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


Now, there are No Null values 

-Next, rounding up to 3 decimal places and multiply the numbers by 100 (to present the percentages)

In [18]:
# Rounding the float values to 3 decimal places
race["White"] = (race["White"].round(3))*100
race["Black"] = (race["Black"].round(3))*100
race["Hispanic"] = (race["Hispanic"].round(3))*100
race["Asian"] = (race["Asian"].round(3))*100
race.head(6)

,Location,White,Black,Hispanic,Asian
0,United States,61.0,12.0,18.0,6.0
1,Alabama,65.0,27.0,4.0,4.6
2,Alaska,58.0,3.0,6.0,7.0
3,Arizona,51.0,4.0,37.0,3.0
4,Arkansas,74.0,15.0,7.0,2.0
5,California,39.0,6.0,38.0,15.0


In [19]:
#renaming Location column 
race.rename(columns={'Location': 'State'}, inplace=True)

## C. Merging the Datasets

### 1. Checking the columns of the datasets to merge 

In [20]:
homelessnessCopy2 = homelessnessCopy1.copy()
race.head()

,State,White,Black,Hispanic,Asian
0,United States,61.0,12.0,18.0,6.0
1,Alabama,65.0,27.0,4.0,4.6
2,Alaska,58.0,3.0,6.0,7.0
3,Arizona,51.0,4.0,37.0,3.0
4,Arkansas,74.0,15.0,7.0,2.0


In [21]:
homelessnessCopy2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
StateAbbrev    54 non-null object
2015           54 non-null int64
State          54 non-null object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [22]:
CostOfLivingCities2015USA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 2 columns):
CostOfLiving    36 non-null float64
StateAbbrev     36 non-null object
dtypes: float64(1), object(1)
memory usage: 656.0+ bytes


### 2. Merging

In [23]:
homelessnessCopy2 = homelessnessCopy2.merge(population, on='State')
homelessnessCopy2 = homelessnessCopy2.merge(race, on='State')

homelessnessCopy2.head()

,StateAbbrev,2015,State,pop2010,pop2011,pop2012,pop2013,pop2014,pop2015,pop2016,White,Black,Hispanic,Asian
0,AK,1208,Alaska,714031,722713,731089,736879,736705,737709,741894,58.0,3.0,6.0,7.0
1,AL,1153,Alabama,4785492,4799918,4815960,4829479,4843214,4853875,4863300,65.0,27.0,4.0,4.6
2,AR,830,Arkansas,2921995,2939493,2950685,2958663,2966912,2977853,2988248,74.0,15.0,7.0,2.0
3,AZ,5631,Arizona,6408312,6467163,6549634,6624617,6719993,6817565,6931071,51.0,4.0,37.0,3.0
4,CA,41174,California,37332685,37676861,38011074,38335203,38680810,38993940,39250017,39.0,6.0,38.0,15.0


In [24]:
CostOfLivingCities2015USA.head()

,CostOfLiving,StateAbbrev
0,100.275779,CA
1,100.000000,NY
2,99.033270,HI
3,97.717446,DC
4,91.836137,AK


In [25]:
homelessnessCopy2.drop(['pop2010','pop2011','pop2012','pop2013','pop2014','pop2016'], axis=1, inplace=True)
homelessnessCopy2.head()

,StateAbbrev,2015,State,pop2015,White,Black,Hispanic,Asian
0,AK,1208,Alaska,737709,58.0,3.0,6.0,7.0
1,AL,1153,Alabama,4853875,65.0,27.0,4.0,4.6
2,AR,830,Arkansas,2977853,74.0,15.0,7.0,2.0
3,AZ,5631,Arizona,6817565,51.0,4.0,37.0,3.0
4,CA,41174,California,38993940,39.0,6.0,38.0,15.0


In [26]:
homelessnessCopy2 = homelessnessCopy2.merge(CostOfLivingCities2015USA, on='StateAbbrev')
homelessnessCopy2.head()

,StateAbbrev,2015,State,pop2015,White,Black,Hispanic,Asian,CostOfLiving
0,AK,1208,Alaska,737709,58.0,3.0,6.0,7.0,91.836137
1,AL,1153,Alabama,4853875,65.0,27.0,4.0,4.6,74.660620
2,AZ,5631,Arizona,6817565,51.0,4.0,37.0,3.0,72.588985
3,CA,41174,California,38993940,39.0,6.0,38.0,15.0,100.275779
4,CO,5335,Colorado,5448819,69.0,4.0,21.0,3.0,75.443766


In [27]:
homelessnessCopy2.head(54)

,StateAbbrev,2015,State,pop2015,White,Black,Hispanic,Asian,CostOfLiving
0,AK,1208,Alaska,737709,58.0,3.0,6.0,7.0,91.836137
1,AL,1153,Alabama,4853875,65.0,27.0,4.0,4.6,74.660620
2,AZ,5631,Arizona,6817565,51.0,4.0,37.0,3.0,72.588985
3,CA,41174,California,38993940,39.0,6.0,38.0,15.0,100.275779
4,CO,5335,Colorado,5448819,69.0,4.0,21.0,3.0,75.443766
5,CT,3134,Connecticut,3584730,70.0,9.0,15.0,4.0,88.098987
6,DC,7298,District of Columbia,670377,37.0,46.0,11.0,5.0,97.717446
7,FL,4152,Florida,20244914,54.0,16.0,26.0,2.0,83.932619
8,GA,5797,Georgia,10199398,52.0,31.0,10.0,5.0,75.506200
9,HI,4903,Hawaii,1425157,19.0,2.0,10.0,39.0,99.033270


In [28]:
# 1. converting Count to String (remove the comma) and then to an integer
homelessnessCopy2['pop2015'] = homelessnessCopy2['pop2015'].str.replace(',', '')
homelessnessCopy2['pop2015'] = pd.to_numeric(homelessnessCopy2['pop2015'])
homelessnessCopy2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 9 columns):
StateAbbrev     36 non-null object
2015            36 non-null int64
State           36 non-null object
pop2015         36 non-null int64
White           36 non-null float64
Black           36 non-null float64
Hispanic        36 non-null float64
Asian           36 non-null float64
CostOfLiving    36 non-null float64
dtypes: float64(5), int64(2), object(2)
memory usage: 2.8+ KB


In [29]:
perc2015 = 100 * (homelessnessCopy2[2015]/ homelessnessCopy2['pop2015']) 
homelessnessCopy2['perc2015'] = perc2015*100
homelessnessCopy2.head(54)

,StateAbbrev,2015,State,pop2015,White,Black,Hispanic,Asian,CostOfLiving,perc2015
0,AK,1208,Alaska,737709,58.0,3.0,6.0,7.0,91.836137,16.375021
1,AL,1153,Alabama,4853875,65.0,27.0,4.0,4.6,74.660620,2.375422
2,AZ,5631,Arizona,6817565,51.0,4.0,37.0,3.0,72.588985,8.259547
3,CA,41174,California,38993940,39.0,6.0,38.0,15.0,100.275779,10.559077
4,CO,5335,Colorado,5448819,69.0,4.0,21.0,3.0,75.443766,9.791113
5,CT,3134,Connecticut,3584730,70.0,9.0,15.0,4.0,88.098987,8.742639
6,DC,7298,District of Columbia,670377,37.0,46.0,11.0,5.0,97.717446,108.864117
7,FL,4152,Florida,20244914,54.0,16.0,26.0,2.0,83.932619,2.050885
8,GA,5797,Georgia,10199398,52.0,31.0,10.0,5.0,75.506200,5.683669
9,HI,4903,Hawaii,1425157,19.0,2.0,10.0,39.0,99.033270,34.403227


## D. Drawing

In [30]:
Colorscl = [[0.0, 'rgb(230, 240, 255)'],[0.2, 'rgb(179, 209, 255)'],
            [0.4, 'rgb(102, 163, 255)'], [0.6, 'rgb(0, 102, 255)'],
            [0.8, 'rgb(0, 71, 179)'],[1.0, 'rgb(0, 31, 77)']]

text = [
    f'{t} <br> Percentage of Homeless: {r:.2f}% <br> Percentage of Black People: {j:.2f}%'
    for t, r,j in zip(homelessnessCopy2['State'], homelessnessCopy2['perc2015'],homelessnessCopy2['Black'])
]

data = [
    go.Scatter(
        x=homelessnessCopy2['perc2015'],
        y=homelessnessCopy2['CostOfLiving'],
        text=text,
        mode='markers',
        marker=dict(
         #   sizemin=5,
            size=15,
            colorscale=Colorscl,
            showscale=True,
            color=homelessnessCopy2['Black'],
            line=dict(color='black', width=1.2)
        ))
]

figure = go.Figure(
    data=data,
    layout=go.Layout(    
        xaxis=dict(title='Percentage of Homeless',
           #        range=[0, 105]
                  ),
        yaxis=dict(title='Cost Of Living',
                       range=[0, 105],
                          dtick=10),
        title='Percentage of Homeless vs Cost Of Living Colored by Percentage of Black Individuals, 2015'
    ))

iplot(figure)